>https://www.youtube.com/watch?v=9aYuQmMJvjA

# Convnet Intro - Convolutional Neural Networks (p. 5)

- <mark>Convolutional Neural Networks (CNN)</mark> mainly used for image tasks; but it's out-performing ***recurrent neural networks*** so it's good in training **sequential** types of data
- In image tasks, it tasks in images in it's raw form - even 3D images. **NO** need to flatten or adjust.
- <mark>Convolutions</mark>: goal is to locate features of a image through its pixels
- Image as input is **called** a <mark>window</mark> or <mark>"X by Y" (dimension) "Convolution Kernel"</mark>
- Convolution first finds edges, curves, then searches a more complex one level up like a shape (multiple edges or curves) and so on...

![alt text](img/ConvolutionConcept.png)

- after convolution, then <mark>pooling</mark>
- <mark>Max Pooling</mark>: algorithm that takes maximum value generated by the convolution process in every window generated, **basically simplifying the image more and more**
- Every level up (layer) of convolution **INCREASES** kernel/window dimension
- underestimated step not in tutorials is the **Preprocessing** step of datasets. this is to prepare it for the neural network and realistically it'll always take a long time.

In [12]:
import os
import cv2
import numpy as np
from tqdm import tqdm

**Cats v Dogs Dataset found here:** https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765

In [16]:
REBUILD_DATA = False

class DogsVSCats():
    # we're going to make images 50 x 50
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                # handle empty images
                except Exception as e:
                    pass
                    #print(str(e))
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print("\nCats: ", self.catcount)
        print("\nDogs: ", self.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()

In [14]:
np.eye(10)[7]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [18]:
training_data = np.load("training_data.npy", allow_pickle = True)

In [19]:
print(len(training_data))

24946
